In [ ]:
import torch
import torch.nn as nn
import numpy as np
import joblib
from torch_geometric.data import Data

In [ ]:
patient_and_stage = joblib.load("esca_patients_and_stage.pkl")

In [ ]:
len(patient_and_stage)

In [ ]:
patient_and_stage['TCGA-LN-A4MR']

In [ ]:
tmp_stage = []
for x in patient_and_stage:
    tmp_stage.append(patient_and_stage[x])
print(set(tmp_stage))

In [ ]:
patient_and_label = {}
for x in patient_and_stage:
    if patient_and_stage[x] in ['Stage III', 'Stage IIIB', 'Stage IIIC', 'Stage IV', 'Stage IVA', 'Stage IIIA']:
        patient_and_label[x] = 1
    elif patient_and_stage[x] in [ 'Stage IIB', 'Stage IA', 'Stage IB', 'Stage II', 'Stage I', 'Stage IIA']:
        patient_and_label[x] = 0

In [ ]:
patient_and_label['TCGA-LN-A4MR']

In [ ]:
patients = []
for x in patient_and_label:
    patients.append(x)

In [ ]:
no_sort_t_all_feature = joblib.load("esca_5x10x_kimianet_patch.pkl")

In [ ]:
#pretrain feature
no_sort_t_all_feature['TCGA-LN-A4MR']['30-26.png']

In [ ]:
t_all_feature = {}
for x in patients:
    feature = {}
    for z in no_sort_t_all_feature[x]:
        if z == '-1':
            feature[z] = no_sort_t_all_feature[x][z]
            
    for z in no_sort_t_all_feature[x]:            
        if len(z.split('-')) == 6:
            feature[z] = no_sort_t_all_feature[x][z]
        
    for z in no_sort_t_all_feature[x]:            
        if len(z.split('-')) == 2 and z != '-1':
            feature[z] = no_sort_t_all_feature[x][z]
        
    t_all_feature[x] = feature


In [ ]:
"""
Thumbnail:"-1"
5x:"x1-x2-y1-y2-x-y",x/y are the coordinates of the 5x node, and x1/x2/y1/y2 are the coordinates of the corresponding 10x node
10x:"x-y"
"""
t_all_feature['TCGA-LN-A4MR']['-1']

In [ ]:
#e.g. The coordinates of this 5x patch in 5x wsi are 17-14, and it consists of four patches with coordinates 34-28, 34-29, 35-28, 35-29 in 10x wsi .
t_all_feature['TCGA-LN-A4MR']['34-35-28-29-17-14.png']

In [ ]:
t_all_feature['TCGA-LN-A4MR']['24-23.png']

In [ ]:
all_feature = {}
for x in patients:
    feature = []
    for z in t_all_feature[x]:
        if z == '-1':
            feature.append(t_all_feature[x][z])
    for z in t_all_feature[x]:            
        if len(z.split('-')) == 6:
            feature.append(t_all_feature[x][z])
    for z in t_all_feature[x]:            
        if len(z.split('-')) == 2 and z != '-1':
            feature.append(t_all_feature[x][z])
        
    all_feature[x] = feature


In [ ]:
all_feature['TCGA-LN-A4MR'][:5]

In [ ]:

def get_edge_index_2(id):  
    
    start = []
    end = []
    
    patch_id = {}
    i=0
    for x in t_all_feature[id]:
        patch_id[x.split('.')[0]] = i
        i+=1
#     print(patch_id)

    # Edges between nodes of different resolutions
    for x in patch_id:
        if len(x.split('-'))==6:
            start.append(patch_id[x])
            end.append(patch_id['-1'])
            end.append(patch_id[x])
            start.append(patch_id['-1']) 
            
            if x.split('-')[0]+'-'+x.split('-')[2] in patch_id:
                start.append(patch_id[x])
                end.append(patch_id[x.split('-')[0]+'-'+x.split('-')[2]])
                end.append(patch_id[x])
                start.append(patch_id[x.split('-')[0]+'-'+x.split('-')[2]])                
            if x.split('-')[1]+'-'+x.split('-')[2] in patch_id:                
                start.append(patch_id[x])
                end.append(patch_id[x.split('-')[1]+'-'+x.split('-')[2]]) 
                end.append(patch_id[x])
                start.append(patch_id[x.split('-')[1]+'-'+x.split('-')[2]])             
            if x.split('-')[0]+'-'+x.split('-')[3] in patch_id:            
                start.append(patch_id[x])
                end.append(patch_id[x.split('-')[0]+'-'+x.split('-')[3]])
                end.append(patch_id[x])
                start.append(patch_id[x.split('-')[0]+'-'+x.split('-')[3]])            
            if x.split('-')[1]+'-'+x.split('-')[3] in patch_id:            
                start.append(patch_id[x])
                end.append(patch_id[x.split('-')[1]+'-'+x.split('-')[3]])
                end.append(patch_id[x])
                start.append(patch_id[x.split('-')[1]+'-'+x.split('-')[3]]) 


        
    patch_id_5x = {}
    for x in patch_id:
        if len(x.split('-'))==6:
            patch_id_5x[x.split('-')[-2]+'-'+x.split('-')[-1]] = patch_id[x]
#     print(patch_id_5x)

    #5x resolution edge between nodes
    for x in patch_id:
        if len(x.split('-'))==6:
            i = int(x.split('-')[-2])
            j = int(x.split('-')[-1])
#             print(i,j)
            if str(i)+'-'+str(j+1) in patch_id_5x:
                start.append(patch_id_5x[str(i)+'-'+str(j)])
                end.append(patch_id_5x[str(i)+'-'+str(j+1)])
            if str(i)+'-'+str(j-1) in patch_id_5x:
                start.append(patch_id_5x[str(i)+'-'+str(j)])
                end.append(patch_id_5x[str(i)+'-'+str(j-1)])
            if str(i+1)+'-'+str(j) in patch_id_5x:
                start.append(patch_id_5x[str(i)+'-'+str(j)])
                end.append(patch_id_5x[str(i+1)+'-'+str(j)])
            if str(i-1)+'-'+str(j) in patch_id_5x:
                start.append(patch_id_5x[str(i)+'-'+str(j)])
                end.append(patch_id_5x[str(i-1)+'-'+str(j)])
            if str(i+1)+'-'+str(j+1) in patch_id_5x:
                start.append(patch_id_5x[str(i)+'-'+str(j)])
                end.append(patch_id_5x[str(i+1)+'-'+str(j+1)])
            if str(i-1)+'-'+str(j+1) in patch_id_5x:
                start.append(patch_id_5x[str(i)+'-'+str(j)])
                end.append(patch_id_5x[str(i-1)+'-'+str(j+1)])
            if str(i+1)+'-'+str(j-1) in patch_id_5x:
                start.append(patch_id_5x[str(i)+'-'+str(j)])
                end.append(patch_id_5x[str(i+1)+'-'+str(j-1)])
            if str(i-1)+'-'+str(j-1) in patch_id_5x:
                start.append(patch_id_5x[str(i)+'-'+str(j)])
                end.append(patch_id_5x[str(i-1)+'-'+str(j-1)])
                
    #10x resolution edge between nodes
    for x in patch_id:
        if len(x.split('-'))==2 and x!='-1':
#             print(x)
            i = int(x.split('-')[0])
            j = int(x.split('-')[1])
#             print(i,j)
            if str(i)+'-'+str(j+1) in patch_id:
                start.append(patch_id[str(i)+'-'+str(j)])
                end.append(patch_id[str(i)+'-'+str(j+1)])
            if str(i)+'-'+str(j-1) in patch_id:
                start.append(patch_id[str(i)+'-'+str(j)])
                end.append(patch_id[str(i)+'-'+str(j-1)])
            if str(i+1)+'-'+str(j) in patch_id:
                start.append(patch_id[str(i)+'-'+str(j)])
                end.append(patch_id[str(i+1)+'-'+str(j)])
            if str(i-1)+'-'+str(j) in patch_id:
                start.append(patch_id[str(i)+'-'+str(j)])
                end.append(patch_id[str(i-1)+'-'+str(j)])
            if str(i+1)+'-'+str(j+1) in patch_id:
                start.append(patch_id[str(i)+'-'+str(j)])
                end.append(patch_id[str(i+1)+'-'+str(j+1)])
            if str(i-1)+'-'+str(j+1) in patch_id:
                start.append(patch_id[str(i)+'-'+str(j)])
                end.append(patch_id[str(i-1)+'-'+str(j+1)])
            if str(i+1)+'-'+str(j-1) in patch_id:
                start.append(patch_id[str(i)+'-'+str(j)])
                end.append(patch_id[str(i+1)+'-'+str(j-1)])
            if str(i-1)+'-'+str(j-1) in patch_id:
                start.append(patch_id[str(i)+'-'+str(j)])
                end.append(patch_id[str(i-1)+'-'+str(j-1)])               
                
                  
    
#     print(start)
#     print(end)
    return [start,end]  

In [ ]:
get_edge_index_2('TCGA-LN-A4MR')[0][:20],get_edge_index_2('TCGA-LN-A4MR')[1][:20]

In [ ]:
all_node_type = {}
for x in patients:
    t = []   
    for z in t_all_feature[x]:
        if z == '-1':
            t.append(0)
    for z in t_all_feature[x]:            
        if len(z.split('-')) == 6:
            t.append(1)
    for z in t_all_feature[x]:            
        if len(z.split('-')) == 2 and z != '-1':
            t.append(2)        
      
    all_node_type[x] = t

In [ ]:
"""
Thumbnail: 0
5x: 1
10x: 2
"""
np.array(all_node_type['TCGA-LN-A4MR'])

In [ ]:
#The first node is the thumbnail, it has no parent, so it is set to -1.
#The following nodes are 5 times, and their parent is the thumbnail, so the index is 0.
#The last is 10 times, respectively, corresponding to their 5 times the index of the parent node.
tree = {}
for p_id in patients:
    
    t_tree = np.array([-2]*len(all_node_type[p_id]))
    t_tree[np.where(np.array(all_node_type[p_id])==0)] = -1
    t_tree[np.where(np.array(all_node_type[p_id])==1)] = np.where(np.array(all_node_type[p_id])==0)[0][0]
#     print(t_tree)
    
    patch_id = {}
    i=0
    for x in t_all_feature[p_id]:
        patch_id[x.split('.')[0]] = i
        i+=1
#     print(patch_id)

    for x in patch_id:
        if len(x.split('-'))==6:
            if x.split('-')[0]+'-'+x.split('-')[2] in patch_id:
                t_tree[patch_id[x.split('-')[0]+'-'+x.split('-')[2]]] =  patch_id[x]           
            if x.split('-')[1]+'-'+x.split('-')[2] in patch_id:                
                t_tree[patch_id[x.split('-')[1]+'-'+x.split('-')[2]]] =  patch_id[x]              
            if x.split('-')[0]+'-'+x.split('-')[3] in patch_id:            
                t_tree[patch_id[x.split('-')[0]+'-'+x.split('-')[3]]] =  patch_id[x]            
            if x.split('-')[1]+'-'+x.split('-')[3] in patch_id:            
                t_tree[patch_id[x.split('-')[1]+'-'+x.split('-')[3]]] =  patch_id[x]   
    

    tree[p_id] = t_tree
#     break

In [ ]:
tree['TCGA-LN-A4MR']

In [ ]:
#Record the coordinates of each patch.
x_y = {}
for x in patients:
    dd = []
    for z in t_all_feature[x]:
        if z == '-1':
            dd.append((0,0))
            
    for z in t_all_feature[x]:            
        if len(z.split('-')) == 6:
            dd.append((float(z.split('-')[-2]),float(z.split('-')[-1].split('.')[0])))
        
    for z in t_all_feature[x]:            
        if len(z.split('-')) == 2 and z != '-1':
            dd.append((float(z.split('-')[0]),float(z.split('-')[1].split('.')[0])))
        
    x_y[x] = dd


In [ ]:
#Get the maximum value of the coordinates for the next step of normalization.
max_x_y_5x = {}
max_x_y_10x = {}
for p_id in patients:
    x=[]
    y=[]
    for z in np.array(x_y[p_id])[np.where(np.array(all_node_type[p_id])==1)]:
        x.append(z[0])
        y.append(z[1])
    max_x_y_5x[p_id] = [np.max(np.array(x)),np.max(np.array(y))]
    
    x=[]
    y=[]
    for z in np.array(x_y[p_id])[np.where(np.array(all_node_type[p_id])==2)]:
        x.append(z[0])
        y.append(z[1])
    max_x_y_10x[p_id] = [np.max(np.array(x)),np.max(np.array(y))]    
    

In [ ]:
#Normalize.
x_y = {}
for x in patients:
    dd = []
    for z in t_all_feature[x]:
        if z == '-1':
            dd.append((0,0))
            
    for z in t_all_feature[x]:            
        if len(z.split('-')) == 6:
            dd.append((float(z.split('-')[-2])/max_x_y_5x[x][0],float(z.split('-')[-1].split('.')[0])/max_x_y_5x[x][1]))
        
    for z in t_all_feature[x]:            
        if len(z.split('-')) == 2 and z != '-1':
            dd.append((float(z.split('-')[0])/max_x_y_10x[x][0],float(z.split('-')[1].split('.')[0])/max_x_y_10x[x][1]))
        
    x_y[x] = dd


In [ ]:
#x_y records the normalized relative position coordinates of all nodes in each heterogeneous graph.
x_y['TCGA-LN-A4MR'][:5]

In [ ]:

all_data = {}
for id in patients:
    print(id)
    node_attr=torch.tensor(all_feature[id],dtype=torch.float)
    edge_index_tree_8nb = torch.tensor(get_edge_index_2(id),dtype=torch.long) 
    batch = torch.tensor([0 for i in range(len(node_attr))])
    node_type = torch.tensor(all_node_type[id])
    node_tree = torch.tensor(tree[id])
    x_y_index = torch.tensor(x_y[id])
    data_id = id 
    data=Data(x=node_attr,edge_index_tree_8nb=edge_index_tree_8nb,data_id=data_id,batch =batch,node_type=node_type,node_tree=node_tree,x_y_index=x_y_index) 
    all_data[id] = data
#     print(data)
    

In [ ]:
all_data['TCGA-LN-A4MR']

In [ ]:
# joblib.dump(all_data,'esca_5x10x_tree_8nb_gnn_data.pkl')